# foo <font color='red'>??-??-?? Data Analysis Notebook</font>**`??-??-?? Data Analysis Notebook`**

## Daily Goals

[Enter Goals Here]

## Daily Summary

[Enter Summary Here]

# `Prelimiaries`

## Globals and Includes

In [1]:
date = "160828"  
directory = "//Andor/share/Data and documents/Data repository/" + date + "/"
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.optimize import curve_fit as fit
from matplotlib.ticker import ScalarFormatter
from matplotlib import gridspec
%matplotlib inline

In [9]:
%%javascript 
// use the javascript language in this cell to control how the notebook itself behaves.
// Effectively Get rid of the scrollbar on the output, just let it be large.
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

## Matplotlib Defaults

In [10]:
# set matplotlib plot defaults :D
# These values are nice for a dark theme.
mpl.rcParams['text.color'] = '#ffffff'
mpl.rcParams['figure.edgecolor'] = '#ffffff'
mpl.rcParams['xtick.color'] = '#ffffff'
mpl.rcParams['ytick.color'] = '#ffffff'
mpl.rcParams['figure.facecolor'] = '#000000'
mpl.rcParams['axes.facecolor'] = '#0a0a0a'
mpl.rcParams['figure.figsize'] = (18.0, 8.0)
mpl.rcParams['axes.labelcolor'] = '#ffffff'
mpl.rcParams['grid.color'] = '#aaaaff'
mpl.rcParams['axes.edgecolor'] = '#ffffff'
mpl.rcParams['legend.facecolor'] = '#0a0a0a'
mpl.rcParams['axes.grid'] = True
mpl.rcParams['axes.formatter.useoffset'] = False

## Fitting Functions

In [2]:
# Fits 
def quadratic(x,a,b,x0):
    # This assumes downward facing. Best to write another function for upward facing if need be, I think.
    if a < 0:
        return 10**10
    if b > 0:
        return 10**10
    return a + b*(x-x0)**2


def gaussian(x, A1, x01, sig1, offset):
    if (offset < 0):
        return 10**10
    return offset + A1 * np.exp(-(x-x01)**2/(2*sig1**2))


def doubleGaussian(x, A1, x01, sig1, A2, x02, sig2, offset):
    if (A1 < 0 or A2 < 0):
        # Penalize negative fits.
        return 10**10
    if (offset < 0):
        return 10**10
    return offset + A1 * np.exp(-(x-x01)**2/(2*sig1**2)) + A2 * np.exp(-(x-x02)**2/(2*sig2**2))


def tripleGaussian(x, A1, x01, sig1, A2, x02, sig2, A3, x03, sig3, offset ):
    if (A1 < 0 or A2 < 0 or A3 < 0):
        # Penalize negative fits.
        return 10**10
    if (offset < 0):
        return 10**10
    return (offset + A1 * np.exp(-(x-x01)**2/(2*sig1**2)) + A2 * np.exp(-(x-x02)**2/(2*sig2**2)) 
            + A3 * np.exp(-(x-x03)**2/(2*sig3**2)))        


def decayingCos(x, A, tau, f, phi, offset):
    # Just for sanity. Keep some numbers positive.
    if (A < 0):
        return x * 10**10
    if (phi < 0):
        return x * 10**10
    if (offset < 0):
        return x * 10**10
    # no growing fits.
    if (tau > 0):
        return x * 10**10
    return offset + (1 - A/2 * np.exp(-x/tau) * np.cos(2 * np.pi * f * x + phi))


def sinc2(x, A, center, scale, offset):
    """
    The 2 here referes to squared!
    """
    if (offset < 0):
        return x * 10**10
    if (A < 0):
        return x * 10**10
    return (A * np.sinc((x - center)/scale)**2 + offset)


def lorentzian(x, A, center, width, offset):
    if (offset < 0):
        return x * 10**10
    if (A < 0):
        return x * 10**10
    return (A /((x - center)**2 + (width/2)**2))


def poissonian(x, k, weight):    
    """
    This function calculates p_k{x} = weight * e^(-k) * k^x / x!.
    :param x: argument of the poissonian
    :param k: order or (approximate) mean of the poissonian.
    :param weight: a weight factor, related to the maximum data this is supposed to be fitted to, but typically over-
    weighted for the purposes of this function.
    :return: the poissonian evaluated at x given the parametes.
    """
    import numpy as np
    term = 1
    # calculate the term k^x / x!. Can't do this directly, x! is too large.
    for n in range(0, int(x)):
        term *= k / (x - n)
    return np.exp(-k) * term * weight


## The Import Data Function

In [ ]:
def importData(date, name, runNumber):
    """
    This function imports the data created by my first-step python code.
    :param date: the date in YYMMDD format so that the code knows where to look for the data.
    :param name: the name of the compiled data file, minus _run?? (e.g. it's standardized for carrier calibrations
    :param runNumber: The run number.
    :return: returns an OrderedDictionary object from the collections library that contains all the information about
             a run.
    """
    from collections import OrderedDict as dic
    data = dic()
    with open('//andor/share/Data and documents/Data repository/' + str(date) + '/' + str(name) + '_run'
              + str(runNumber) + '.csv') as myfile:
        fileText = myfile.read().replace('\n', '')
        rows = []
        for row in fileText.split(':'):
            rows.append(row.replace('\t', ''))
        del rows[0]
        for itemInc in range(0, len(rows), 2):
            if (rows[itemInc + 1].find(';')) != -1:
                itemData = dic()
                subrows = []
                for subrow in rows[itemInc + 1].split(';'):
                    subrows.append(subrow)
                del subrows[0]
                for subitem in range(0, len(subrows), 2):
                    try:
                        itemData[subrows[subitem]] = [float(item) for item in (subrows[subitem + 1].split(','))]
                    except:
                        itemData[subrows[subitem]] = subrows[subitem + 1].split(',')

                data[rows[itemInc]] = itemData
            else:
                try:
                    data[rows[itemInc]] = [float(item) for item in (rows[itemInc + 1].split(','))]
                except:
                    data[rows[itemInc]] = rows[itemInc + 1].split(',')
    return data

## standardSurvivalPlot(data, locs)

In [1]:
def standardSurvivalPlot(data, locs):
    """
    This function creates a standard plot. It plots survival data, survival fits, data counts, and loading probabilities.
    It does all of this in a 12x16 gridspec and leaves one corner open for specializing a given plot.
    """
    # Make colormap. really only need len(locs) + 1 rgbs, but adding an extra makes the spacing of the colors
    # on this colormap more sensible.
    cmapRGB = mpl.cm.get_cmap('gist_rainbow', len(locs) + 2)
    colormap = []
    for colorInc in range(cmapRGB.N):
        colormap.append(mpl.colors.rgb2hex(cmapRGB(colorInc)[:3]))
    # Setup grid
    grid1 = mpl.gridspec.GridSpec(12, 16)
    grid1.update(left=0.05, right=0.95, wspace=1.2, hspace=1000)
    gridLeft = mpl.gridspec.GridSpec(12, 16)
    gridLeft.update(left=0.001, right=0.95, hspace=1000)
    gridRight = mpl.gridspec.GridSpec(12, 16)
    gridRight.update(left=0.2, right=0.946, wspace=0, hspace=1000)
    # ### Main Plot
    mainPlot = plt.subplot(grid1[:, :12])
    # Plot Atoms
    for atomInc in range(0, len(locs)):
        mainPlot.errorbar(data['Key'], data[locs[atomInc]]['Survival Averages'], yerr=data[locs[atomInc]]['Survival Errors'],
                          ls='', marker='o', color=colormap[atomInc], capsize=6, elinewidth=3)
        mainPlot.plot(data['xForFit'], data[locs[atomInc]]['fitData'], color=colormap[atomInc], label=locs[atomInc], 
                      linewidth=3)
    # Plot Average
    mainPlot.plot(data['xForFit'], data['[Correlation Averages]']['fitData'], color=colormap[len(locs)], 
                  label='Average', linewidth=3)                                       
    mainPlot.errorbar(data['Key'], data['[Correlation Averages]']['Total average single atom survival'], 
                      yerr=data['[Correlation Errors]']['Total average single atom survival'],
                      ls='', marker='o', color=colormap[len(locs)], capsize=6, elinewidth=3)
    mainPlot.set_ylim({-0.02, 1.01})
    mainPlot.set_xlim({min(data['Key']) - (max(data['Key']) - min(data['Key'])) / len(data['Key']), 
                      max(data['Key']) + (max(data['Key']) - min(data['Key'])) / len(data['Key'])})
    mainPlot.set_xticks(data['Key'])
    mainPlot.set_title("Survival Data", fontsize=30)
    mainPlot.set_ylabel("Survival Probability", fontsize=20)
    mainPlot.set_xlabel("Key Values", fontsize=20)
    mainPlot.legend(loc="upper center", bbox_to_anchor=(0.5,-0.1), fancybox=True, ncol=4)
    mainPlot.grid("on")
    # ### Capture Probabilities Plot
    capturePlot = plt.subplot(grid1[8:12, 12:16])
    for atomInc in range(len(locs)):
        capturePlot.plot(data['Key'], data[locs[atomInc]]['Loading Probabilities'], ls='', marker='o', 
                         color=colormap[atomInc])
        capturePlot.axhline(np.mean(data[locs[atomInc]]['Loading Probabilities']), color=colormap[atomInc])
    capturePlot.set_ylim({0, 1})
    capturePlot.set_xlim({min(data['Key']) - (max(data['Key']) - min(data['Key'])) / len(data['Key']), 
                      max(data['Key']) + (max(data['Key']) - min(data['Key'])) / len(data['Key'])})
    capturePlot.set_xlabel("Key Values")
    capturePlot.set_ylabel("Capture %")
    capturePlot.set_title("Capture Probabilities")
    capturePlot.grid("on")
    # ### Count Series Plot
    countDataPlot = plt.subplot(gridRight[4:8, 12:15])
    for atomInc in range(len(locs)):
        countDataPlot.plot(data[locs[atomInc]]['Data Counts'], color=colormap[atomInc], ls='', marker='.', markersize=1)
        countDataPlot.axhline(data[locs[atomInc]]['Threshold'], color=colormap[atomInc])
    countDataPlot.set_xlabel("Picture #")
    countDataPlot.set_ylabel("Camera Signal")
    countDataPlot.set_title("Camera Signal Over Time")
    countDataPlot.grid("on")
    ticksForVis = countDataPlot.xaxis.get_major_ticks()
    ticksForVis[-1].label1.set_visible(False)
    # Count Histogram Plot
    countHistPlot = plt.subplot(gridLeft[4:8, 15:16], sharey=countDataPlot)
    # countHistPlot = plt.subplot2grid((16, 16), (4, 15), rowspan=4, sharey=countDataPlot)
    for atomInc in range(len(locs)):
        countHistPlot.hist(data[locs[atomInc]]["Data Counts"], 50, color=colormap[atomInc], orientation='horizontal',
                           alpha=1/len(locs))
        countHistPlot.axhline(data[locs[atomInc]]["Threshold"], color=colormap[atomInc])
    ticks = countHistPlot.get_xticklabels()
    for tickInc in range(len(ticks)):
        ticks[tickInc].set_rotation(-45)
    plt.setp(countHistPlot.get_yticklabels(), visible=False)

## Data Structure Usage

Current Base Key Strings (data['???']):  
- Key List
- Date
- Run Number
- Repetitions
- Pictures Per Experiment
- Key
- < Atom Locations >
- Correlation Averages
- Correlation Errors

Current Atom Key Strings (e.g. data['[3, 1]']['???'])
- Key List
- Survival Averages
- Survival Errors
- Loading Probabilities
- Atom Location
- Threshold
- Threshold Fidelity
- Atoms Data

Current Multi-Atom Key Strings (e.g. data['[Correlation Averages]']['???'])
- Load < 1-max >, atom < 0-max-1 > survived
- Load < 1-max >, < 0-max > atoms survived
- Load < 1-max >, average single atom survival
- Total average single atom survival

Similar for Multi-Atom Errors.

## Markdown Syntax Quick Guide

Markdown | Table | Example
--- | :---: | ---:
*It* | `renders` | **nicely**
~~1~~ | 2 | 3

```str hello = 'world'
errBox("not an *error*")
return FALSE;```

> what is going
> on

---

- Lists
+ Are
* Fun  


***

1.  stuff
  * Unordered sub-list.
1000. Actual numbers don't matter, just that it's a number
+ I lied, just the first bullet for a given list determines the type.
  1. Ordered sub-list  
    hey.
    2. what?

___

html and links are also possible.

# `Data`